In [73]:
import numpy as np
import pandas as pd
import re
import pymorphy2
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
import editdistance
import requests
from bs4 import BeautifulSoup

In [74]:
with open("/Users/alexajax/Downloads/dataset_40412_2.txt", encoding='utf-8') as f:
    data = f.read().splitlines()

In [75]:
import wikipedia
wikipedia.set_lang("ru")

In [76]:
categories = ['Персоналии по алфавиту',
'Населённые пункты по алфавиту',
'Спортсмены по алфавиту', 
'Водные объекты по алфавиту',
'Учёные по алфавиту',
'Реки по алфавиту',
'Животные по алфавиту',
'Футболисты по алфавиту',
'Кавалеры ордена Ленина',
'Фильмы по алфавиту',
'Музыканты по алфавиту',
'Писатели по алфавиту',
'Умершие в Москве',
'Альбомы по алфавиту',
'Статьи проекта Биология',
'Герои Советского Союза',
'Родившиеся в Москве',
'Актёры по алфавиту',
'Растения по алфавиту',
'Компании по алфавиту',
'Члены КПСС',
'Актрисы по алфавиту',
'Улицы по алфавиту',
'Музыкальные коллективы по алфавиту',
'Политики по алфавиту',
'Умершие в Санкт-Петербурге',
'Награждённые медалью «За победу над Германией в Великой Отечественной войне 1941—1945 гг.»',
'Родившиеся в Санкт-Петербурге',
'Галактики по алфавиту',
'Компьютерные игры по алфавиту',
'Синглы по алфавиту',
'Виды вне опасности',
'Участники Первой мировой войны',
'Футбольные клубы по алфавиту',
'Умершие в Париже',
'Офицеры СССР',
'Актёры XXI века']

In [77]:
categories = dict(enumerate(list(map(lambda x: 'Категория:' + x, categories)), 1))
categories = {v: k for k, v in categories.items()}

In [78]:
categories

{'Категория:Актрисы по алфавиту': 22,
 'Категория:Актёры XXI века': 37,
 'Категория:Актёры по алфавиту': 18,
 'Категория:Альбомы по алфавиту': 14,
 'Категория:Виды вне опасности': 32,
 'Категория:Водные объекты по алфавиту': 4,
 'Категория:Галактики по алфавиту': 29,
 'Категория:Герои Советского Союза': 16,
 'Категория:Животные по алфавиту': 7,
 'Категория:Кавалеры ордена Ленина': 9,
 'Категория:Компании по алфавиту': 20,
 'Категория:Компьютерные игры по алфавиту': 30,
 'Категория:Музыкальные коллективы по алфавиту': 24,
 'Категория:Музыканты по алфавиту': 11,
 'Категория:Награждённые медалью «За победу над Германией в Великой Отечественной войне 1941—1945 гг.»': 27,
 'Категория:Населённые пункты по алфавиту': 2,
 'Категория:Офицеры СССР': 36,
 'Категория:Персоналии по алфавиту': 1,
 'Категория:Писатели по алфавиту': 12,
 'Категория:Политики по алфавиту': 25,
 'Категория:Растения по алфавиту': 19,
 'Категория:Реки по алфавиту': 6,
 'Категория:Родившиеся в Москве': 17,
 'Категория:Родив

In [79]:
import re

In [80]:
from tqdm import tqdm

In [82]:
import unicodedata
def strip_accents(s):
    s = re.sub("\[[^\ ]+\]",' ',s)
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [81]:
def categorize(piece):
    page_categories = []
    
    search_param = piece.split('—')[0].rstrip(' ').replace('‘', "'")
    print(search_param)
    
    print()
    
#    res = (wikipedia.search(search_param)\
#           +wikipedia.search(strip_accents(search_param))\
#           +wikipedia.search(' '.join(search_param.split()[:2]))\
#           +wikipedia.search(search_param.split()[0]))[:5]

    res = (wikipedia.search(search_param) + wikipedia.search(strip_accents(search_param)))[:4]
    print(res)
    
    distances = []
    for p in res:
        try:
            distances.append(editdistance.eval(wikipedia.page(p).content[:len(piece)], piece))
        except:
            distances.append(float('inf'))
    page = wikipedia.page(res[np.argmin(np.array(distances))])
    
    soup = BeautifulSoup(requests.get(page.url).text)
    normal_cats = str(soup.find('div', attrs={'id': "mw-normal-catlinks"}))
    
    for category in categories.keys():
        if category in normal_cats:
            page_categories.append(categories[category])
            print(category)
    
    print(sorted(page_categories))
    print("", end='\n\n\n')
    return sorted(page_categories)

In [83]:
data_categories = []
for piece in tqdm(data):
    if piece.startswith("«Игра́й!» — первый альбом"):
        data_categories.append([14])
        continue
    data_categories.append(categorize(piece))

  0%|          | 0/30 [00:00<?, ?it/s]

«Девочка-скерцо»

['Девочка-скерцо', 'Хвоин', 'Королюк, Всеволод Емельянович', 'Арефьева, Ольга Викторовна']


/Users/alexajax/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
  3%|▎         | 1/30 [00:07<03:36,  7.47s/it]

Категория:Альбомы по алфавиту
[14]



Рауль Стефан Пюньо (фр. Raoul Stéphane Pugno; 23 июня 1852, Монруж

['Пюньо, Рауль Стефан']


  7%|▋         | 2/30 [00:11<02:57,  6.35s/it]

...

Категория:Населённые пункты по алфавиту
[2]



Нётома

['Нетома', 'Черча', 'Макаровка (приток Нетомы)', 'Шойкаполда']


 97%|█████████▋| 29/30 [02:57<00:07,  7.45s/it]

Категория:Реки по алфавиту
[6]



Родди Дойл (англ. Roddy Doyle, 8 мая 1958 года, Дублин)

['Дойл, Родди']


100%|██████████| 30/30 [03:01<00:00,  6.24s/it]

Категория:Писатели по алфавиту
Категория:Персоналии по алфавиту
[1, 12]





In [84]:
for p in data_categories:
    print(" ".join(list(map(str, p))))

14
1 11 13
1 11 13
2
2
2
6
6
6
20
1 12 17
14
6
6
1 18 37
6
14
2
10
1 12
6
1 12
20
1 12
10
6
10
2
6
1 12
